In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", None)

In [ ]:
def mean(df):
    x = df.to_numpy()
    return np.nanmean(x, axis=1)

def get_angles(df):
    sensor_azimuth = mean(df.filter(regex='B\d_sensor_azimuth'))
    sensor_zenith = mean(df.filter(regex='B\d_sensor_zenith'))
    solar_azimuth = mean(df.filter(regex='B\d_solar_azimuth'))
    solar_zenith = mean(df.filter(regex='B\d_solar_zenith'))
    rows = np.column_stack((solar_zenith, solar_azimuth, sensor_zenith, sensor_azimuth))
    return pd.DataFrame(data=rows, columns=['sensor_azimuth', 'sensor_zenith', 'solar_azimuth', 'solar_zenith'])

def convert_angles(df):
    tts = df['solar_zenith']
    tto = df['sensor_zenith']
    phi = np.abs((df['solar_azimuth']+180) - (df['sensor_azimuth']+180))
    return pd.DataFrame({ 'tts': tts, 'tto': tto, 'phi': phi })

In [ ]:
df = pd.read_csv("./data_with_angles.csv", index_col=0)
df.reset_index(inplace=True)

angles = convert_angles(get_angles(df))
bands = df.filter(regex='B[123457]$') / 10_000

df = df.drop(df.filter(regex='(zenith|azimuth)$').columns, axis=1)
df = df.drop(df.filter(regex='B[123457]$').columns, axis=1)
df = df.drop(df.filter(regex='(B6|B10)$').columns, axis=1)

df = df.join(bands).join(angles)

In [ ]:
df = df[df['SensorType'] == 'ETM+']
df = df.dropna(subset=['h_grass', 'c_grass'])
df.to_csv("./data_with_angles_etm.csv") 
df = df.groupby(["S_DateNR", "TNAME","TNUM"]).mean()
df = df.reset_index()
df[:10]

In [ ]:
df['B1']

In [ ]:
df.to_csv("./data_with_angles_etm_grouped.csv")

In [ ]:
(df.filter(regex='^B') / 10_000).describe()

In [ ]:
df2 = pd.read_csv("./calculated_prosail_combined.csv", index_col=0)

In [ ]:
df2.filter(regex='^B').describe()